#Web scraping 

In [30]:
!pip install lxml requests beautifulsoup4
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
from bs4 import BeautifulSoup
import requests

In [32]:
import pandas as pd

In [33]:
url_rss = 'https://search.cnbc.com/rs/search/combinedcms/view.xml?partnerId=wrss01&id=10000664'

In [34]:
resp = requests.get(url_rss)
soup = BeautifulSoup(resp.content, features='xml')

In [35]:
items = soup.find_all('item')

In [36]:
len(items)

30

In [37]:
print(items[0].prettify())

<item>
 <link>
  https://www.cnbc.com/2023/05/13/natural-gas-prices-are-bottoming-major-investor-suggests.html
 </link>
 <guid isPermaLink="false">
  107239115
 </guid>
 <metadata:type>
  cnbcnewsstory
 </metadata:type>
 <metadata:id>
  107239115
 </metadata:id>
 <metadata:sponsored>
  false
 </metadata:sponsored>
 <title>
  Major natural gas investor sees signs prices are bottoming
 </title>
 <description>
  U.S. Commodity Funds CEO John Love, who ran the biggest natural gas ETF in the country, cites unusual production dynamics.
 </description>
 <pubDate>
  Sat, 13 May 2023 19:10:14 GMT
 </pubDate>
</item>



In [38]:
# creo una variabile vuota da riempire
news_items = []

In [39]:
for item in items:
  news_item = {}
  news_item['title'] = item.title.text
  news_item['description'] = item.description.text
  news_item['link'] = item.link.text
  news_item['pubDate'] = item.pubDate.text
  news_items.append(news_item)

AttributeError: 'NoneType' object has no attribute 'text'

In [40]:
news_items[0]

{'title': 'Major natural gas investor sees signs prices are bottoming',
 'description': 'U.S. Commodity Funds CEO John Love, who ran the biggest natural gas ETF in the country, cites unusual production dynamics.',
 'link': 'https://www.cnbc.com/2023/05/13/natural-gas-prices-are-bottoming-major-investor-suggests.html',
 'pubDate': 'Sat, 13 May 2023 19:10:14 GMT'}

In [41]:
df = pd.DataFrame(news_items, columns=['title','description','link', 'pubDate'])

In [42]:
df.head()

,title,description,link,pubDate
0,Major natural gas investor sees signs prices a...,"U.S. Commodity Funds CEO John Love, who ran th...",https://www.cnbc.com/2023/05/13/natural-gas-pr...,"Sat, 13 May 2023 19:10:14 GMT"
1,Tesla stock dips after Musk names new Twitter ...,Pros on CNBC discuss Tesla's stock after its C...,https://www.cnbc.com/2023/05/12/tesla-stock-di...,"Fri, 12 May 2023 21:30:16 GMT"
2,Stocks making the biggest moves midday: First ...,These are the stocks posting the largest moves...,https://www.cnbc.com/2023/05/12/stocks-making-...,"Fri, 12 May 2023 21:03:26 GMT"
3,Fed Governor Philip Jefferson named as new vic...,Federal Reserve Governor Philip Jefferson will...,https://www.cnbc.com/2023/05/12/fed-governor-p...,"Fri, 12 May 2023 18:07:04 GMT"
4,Goldman Sachs created an A.I.-powered social m...,"Under CEO David Solomon, Goldman has sought to...",https://www.cnbc.com/2023/05/12/goldman-sachs-...,"Fri, 12 May 2023 18:03:55 GMT"


L'idea da qui ora è quella di aprire il link e leggere l'articolo, partiamo leggendo solo il primo articolo

In [43]:
art0 = df.iloc[17]

In [44]:
art0.pubDate

'Thu, 11 May 2023 18:57:23 GMT'

In [45]:
#estraggo la pagina
art0_link = art0.link

In [46]:
art0_link

'https://www.cnbc.com/2023/05/10/twilio-tanks-on-2q-forecast-this-is-what-it-could-mean-for-the-stock.html'

In [47]:
resp = requests.get(art0_link)
soup = BeautifulSoup(resp.content, features='lxml')

In [48]:
print(soup.title)

<title itemprop="name">Twilio tanks on 2Q forecast. This is what it could mean for the stock</title>


In [49]:
#estraggo il div con class .group
div_group =  soup.find_all("div", attrs={'class':'group'})

In [50]:
type(div_group), len(div_group)

(bs4.element.ResultSet, 0)

In [51]:
inlink = '//www.cnbc.com/quotes/'

In [52]:
inlink = '/quotes/'

In [53]:
# test
link_quote = div_group[10].select('p')[1].select('a')[0]['href']

# estraggo la quotazione dal link
if inlink in link_quote:
  print(link_quote.split('/')[-1])

IndexError: list index out of range

La funzione sottostante estrae tutti i stock ticker di un div, quindi prende tutti i suoi paragrafi ed estrae come prima cosa gli elementi con tag a, che sono sempre attorno a uno stock e hanno il link diretto verso la quotazione. Poi estraggo l'href del tag, se matcha con il link che porta verso le quotazioni, allora l'ultimo elemento sarà lo stock ticker

In [54]:
def extract_ticker(ps):
  stocks = []
  for p in ps:
    links = p.select('a')
    for link in links:
      print(link.get('href'))
      href_link = link.get('href')
      if inlink in href_link:
        stock = href_link.split('/')[-1]
        stocks.append(stock)
  
  return stocks

In [55]:
# estraggo dai div i paragrafi
i = 0
for div in div_group:
  print(f'div {i}')
  p = div.select('p')
  print(f'contiene {len(p)} paragrafi')
  stocks_in_p = extract_ticker(p)
  print(f'Estratti {len(stocks_in_p)} {stocks_in_p}')
  i += 1

Ho estratto tutti i div e ora da ogni paragrafo del div devo estrarre le informazioni necessarie per poi poter aprire delle posizioni di trading sui titoli citati negli articoli

In [56]:
def extract_ticker_from_single_paragraph(p):
  '''
  return the list of stocks present in the paragraph
  '''
  stocks_list = []
  links = p.select('a')
  for link in links:
    href_link = link.get('href')
    if inlink in href_link:
      stock = href_link.split('/')[-1]
      stocks_list.append(stock)
  
  return stocks_list

In [57]:
divs_list = {}

In [58]:
i = 0
# estraggo i div singolarmente
for div in div_group:
  stock_list = []
  #estraggo i paragrafi dal div
  ps = div.select('p')
  j = 0
  # dai paragrafi estraggo gli stock
  for p in ps:
    stock = extract_ticker_from_single_paragraph(p)
    stock_list.append(stock)
    j+=1


In [59]:
divs_list['0'] = stock_list

NameError: name 'stock_list' is not defined

In [60]:
stock_list

NameError: name 'stock_list' is not defined

In [61]:
df_div = pd.DataFrame(divs_list, columns=['stock'])

In [62]:
df_div

,stock


Ora creiamo una funzione che dato il link dell'articolo mi estragga direttamente il tutto

In [63]:
def get_stocks_from_article(link_article):
  # richiedo la pagina dal web
  resp = requests.get(link_article)
  soup = BeautifulSoup(resp.content, features='lxml')

  # estraggo i div con class .group
  div_group =  soup.find_all("div", attrs={'class':'group'})

  # estraggo dal div i paragrafi
  i = 0
  # estraggo i div singolarmente
  for div in div_group:
    stock_list = []
    #estraggo i paragrafi dal div
    ps = div.select('p')
    j = 0
    # dai paragrafi estraggo gli stock
    for p in ps:
      stock = extract_ticker_from_single_paragraph(p)
      stock_list.append(stock)
      j+=1 

  return stock_list


In [64]:
# estraggo gli stock di cui si è parlato nei ultimi 30 articoli

for i in df.index:
  print(f'Articolo {i}')
  print(get_stocks_from_article(df.iloc[i].link))
  print()

Articolo 0
[[], [''], [], [''], [], [], [], [], [], []]

Articolo 1


UnboundLocalError: local variable 'stock_list' referenced before assignment

In [ ]:
df.head()

#Aggiungo la sentiment analysis per ogni paragrafo

##Installazione del necessario

In [ ]:
!pip install transformers[sentencepiece]
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [65]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

NameError: name 'BertForSequenceClassification' is not defined

##Implemento la sentiment analysis
Da ogni Articolo dal quale estraggo i paragrafi voglio avere una sentiment analysis di ogni paragrago, visto che solitamente i paragarafi si riferiscono a un'azione, quindi adesso voglio avere come output **[(['AAPL'],'positive',0.95)]** 

In [66]:
#test sulla sentiment analysis di un paragrafo 
link = df.iloc[2].link
resp = requests.get(link)
soup = BeautifulSoup(resp.content, features='lxml')

# estraggo i div con class .group
div_group =  soup.find_all("div", attrs={'class':'group'})

#prendo il primo paragrafo
par = div_group[1].select('p')[1]
#eseguo la sentiment analysis
result = nlp(par.text)

IndexError: list index out of range

In [ ]:
df.head()

In [ ]:
link = df.iloc[1].link
resp = requests.get(link)
soup = BeautifulSoup(resp.content, features='lxml')

# estraggo i div con class .group
div_group =  soup.find_all("div", attrs={'class':'group'})

par = div_group[0].select('p')[1]

paragraph_analysis(par)

In [ ]:
# funzione che esegue la sentiment analysis
def setiment_analysis_paragraph(p):
  results = nlp(p)
  return results

In [ ]:
# funzione che dato il paragrafo mi estragga gli stock ticker presenti e faccia la sentiment analysis
def paragraph_analysis(paragraph):
  stocks = extract_ticker_from_single_paragraph(paragraph)
  result = nlp(paragraph.text)
  print(f'({stocks}, {result})')
  return (stocks, result[0]['label'], result[0]['score'])

In [67]:
def getDivsFromArticle(article_link):
  resp = requests.get(article_link)
  soup = BeautifulSoup(resp.content, features='lxml')
  div_group = soup.find_all("div", attrs={'class':'group'})

  return div_group

In [68]:
def getParFromDiv(div):
  return div.select('p')

In [69]:
df.index

RangeIndex(start=0, stop=21, step=1)

In [70]:
# eseguo ora la sentiment analysis su ogni paragrafo di ogni articolo
for i in df.index:
  article_link = df.iloc[i].link
  divs_in_article = getDivsFromArticle(article_link)

  
  print(f'publication date: {df.iloc[i].pubDate}')
  j = 0
  for div in divs_in_article:
    pars = getParFromDiv(div)
    print(f'---- Div {j} ----')
    j+=1

    k = 0
    for p in pars:
      print(f'paragraph {k}')
      res = paragraph_analysis(p)
      k+=1
    print()
print()
print('------------------------------------------')


publication date: Sat, 13 May 2023 19:10:14 GMT
---- Div 0 ----
paragraph 0


NameError: name 'paragraph_analysis' is not defined